### 1. Keras笑脸识别
本部分的任务是读取图片并识别图片上的**笑脸**。
Keras是一个比Tensorflow层次更高的框架，一般来说通用的模型都可以通过Keras实现，但是一些复杂的模型在keras中没有，只能用较低层次的Tensorflow或pytorch实现
#### 1.1 引入库 and 加载数据集

In [10]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import kt_utils 

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

# 加载数据
X_train_orig,Y_train_orig,X_test,Y_test,classes = kt_utils.load_dataset()
X_train = X_train_orig / 255
Y_train = Y_train_orig / 255

Y_train = Y_train_orig.T
Y_test = Y_test.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


#### 1.2 使用keras框架构建训练模型
keras与tensorflow不一样的是，我们不需要创建如Z1,A1等的中间变量，只需要保存最后结果，所以在fp的时候只有一个tensor X。

In [12]:
def model(input_shape):
    """
    参数：
        input_shape - 输入参数的size
    返回：
        model - keras模型
    """
    # 类似于tensorflow中的创建占位符
    X_input = Input(input_shape)
    
    # zero padding
    # 由于后面我们要用到7*7的卷积 所以这里p=(7-1)/2=3
    X = ZeroPadding2D((3,3))(X_input)
    
    # Conv - BN - relu
    X = Conv2D(32,(7,7),strides=(1,1),name="conv0")(X)
    X = BatchNormalization(axis=3,name="bn0")(X)
    X = Activation('relu')(X)
    
    # MaxPooling
    X = MaxPooling2D((2,2),name="max_pool")(X)
    
    # 向量化
    X = Flatten()(X)
    # Fully Connected
    X = Dense(1,activation="sigmoid",name="fc")(X)
    
    model = Model(inputs = X_input, outputs = X, name="HappyModel")
    
    return model

In [14]:
# 获取模型
HappyModel = model(X_train.shape[1:])

# 编译模型
HappyModel.compile("adam","binary_crossentropy",metrics=['accuracy'])

# 训练模型
HappyModel.fit(X_train,Y_train,epochs=60,batch_size=50)

# 评估模型
preds = HappyModel.evaluate(X_test,Y_test,batch_size=32,verbose=1,sample_weight=None)
print("Error:"+str(preds[0]))
print("Accuracy:"+str(preds[1]))

Epoch 1/60
600/600 [==============================] - 10s 16ms/step - loss: 2.0804 - acc: 0.5817
Epoch 2/60
600/600 [==============================] - 10s 17ms/step - loss: 0.6351 - acc: 0.7817
Epoch 3/60
600/600 [==============================] - 10s 17ms/step - loss: 0.3815 - acc: 0.8483
Epoch 4/60
600/600 [==============================] - 11s 18ms/step - loss: 0.3199 - acc: 0.8850
Epoch 5/60
600/600 [==============================] - 10s 17ms/step - loss: 0.2238 - acc: 0.9200
Epoch 6/60
600/600 [==============================] - 11s 18ms/step - loss: 0.1241 - acc: 0.9600
Epoch 7/60
600/600 [==============================] - 11s 18ms/step - loss: 0.1171 - acc: 0.9633
Epoch 8/60
600/600 [==============================] - 11s 18ms/step - loss: 0.0948 - acc: 0.9650
Epoch 9/60
600/600 [==============================] - 11s 18ms/step - loss: 0.0899 - acc: 0.9717
Epoch 10/60
600/600 [==============================] - 11s 18ms/step - loss: 0.0709 - acc: 0.9817
Epoch 11/60
600/600 [========

In [15]:
# 利用summary函数观察网络结构
HappyModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 70, 70, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 32)        4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 64, 64, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
__________